This is made for testing Momentum Strategies. First, let's install dependencies


In [3]:
import pandas as pd
import numpy as np
import datetime as dt
from scipy import stats
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
import wrds

Next, let's connect to WRDS

In [2]:
conn = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [5]:
msf = conn.raw_sql("""
                      select a.permno, a.date, 
                      a.ret, a.shrout, a.prc 
                      from crsp.msf as a
                      where a.date >= '12/01/1999'
                      """, date_cols=['date'])

msf = msf[(msf['prc'].notna()) & (msf['ret'].notna()) & (msf['shrout'].notna())]

msf['permno'] = msf['permno'].astype(int)
msf['size'] = msf['shrout'] * msf['prc'].abs()
msf['year'] = msf['date'].dt.year
msf['month'] = msf['date'].dt.month

# create msf_dec
msf_dec = msf[msf['month']==12][['date','permno','year','size']]

# create msf_ls
msf_ls = msf.sort_values(['permno', 'date'])
msf_ls['year_prev'] = msf_ls['year']-1
msf_ls['size_lag'] = msf_ls.groupby('permno')['size'].shift(1)
msf_ls['size_lag'] = np.where(msf_ls['size_lag'].isna(),\
msf_ls['size']/(1+msf_ls['ret']), msf_ls['size_lag'])



See whats in MSF

In [6]:
print(msf)

        permno       date       ret     shrout        prc          size  year  \
0        10001 1999-12-31 -0.004188     2450.0   -8.50000  2.082500e+04  1999   
1        10002 1999-12-31 -0.123036     7729.0   12.18750  9.419719e+04  1999   
2        10009 1999-12-31 -0.202703     2307.0   14.75000  3.402825e+04  1999   
3        10012 1999-12-31  0.107759    19274.0    8.03125  1.547943e+05  1999   
4        10016 1999-12-31 -0.084849    17869.0   18.87500  3.372774e+05  1999   
...        ...        ...       ...        ...        ...           ...   ...   
176674   93426 2023-12-29  0.117416    12510.0   34.07000  4.262157e+05  2023   
176675   93427 2023-12-29  0.175602    36331.0  190.33000  6.914879e+06  2023   
176676   93429 2023-12-29 -0.019924   105556.0  178.56000  1.884808e+07  2023   
176677   93434 2023-12-29  0.065449    43039.0    0.70000  3.012730e+04  2023   
176678   93436 2023-12-29  0.034988  3178921.0  248.48000  7.898983e+08  2023   

        month  
0          